In [2]:
import numpy as np
import pandas as pd
import os
import textwrap
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.linear_model
import sklearn.model_selection
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer, roc_auc_score

In [3]:
data_dir = 'data_readinglevel'
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

N, n_cols = x_train_df.shape
print("Shape of x_train_df: (%d, %d)" % (N, n_cols))
print("Shape of y_train_df: %s" % str(y_train_df.shape))
print(x_train_df["word_count"])

Shape of x_train_df: (5557, 32)
Shape of y_train_df: (5557, 5)
0       119.0
1        61.0
2       133.0
3       118.0
4        80.0
        ...  
5552     89.0
5553     94.0
5554     91.0
5555     99.0
5556     96.0
Name: word_count, Length: 5557, dtype: float64


In [4]:
y_train_df['stage_encoded'] = y_train_df['Coarse Label'].map({'Key Stage 2-3': 0, 'Key Stage 4-5': 1})


In [5]:
y_train_clean = y_train_df['stage_encoded']
random_state=42
zero=0
for i in y_train_clean:
    if i==0:zero+=1
print(zero)

2509


In [6]:
tr_text_list = x_train_df['text'].values.tolist()
bow_preprocessor = CountVectorizer()
X = bow_preprocessor.fit_transform(tr_text_list)

In [ ]:
# def custom_preprocessor(text):
#     text = re.sub(r"_", " ", text)  # Replace underscores with spaces
#     text = re.sub(r"\d+", "", text)  # Remove numbers
#     text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
#     return text.lower().strip()  # Convert to lowercase and trim space
 
# vectorizer = TfidfVectorizer(preprocessor=custom_preprocessor)
my_bow_classifier_pipeline = sklearn.pipeline.Pipeline([
    ('bow', CountVectorizer(min_df=1,max_df=1, ngram_range=(1,1),stop_words=None,token_pattern = r'(?u)\b[\w-]+\b',lowercase = False)),
    ('class', sklearn.linear_model.LogisticRegression(C=1.0, max_iter=200, random_state=random_state)),
])
my_parameter_grid_by_name = dict()
my_parameter_grid_by_name['bow__min_df'] = [1, 5, 10, 0.5, 0.75]
my_parameter_grid_by_name['bow__max_df'] = [0.25, 0.5, 0.75,1.0]
my_parameter_grid_by_name['bow__ngram_range'] = [(1,1),(1,2)]
my_parameter_grid_by_name['bow__stop_words'] = [None,'english']
my_parameter_grid_by_name['bow__token_pattern'] = [r'(?u)\b[\w-]+\b',r'(?u)\b\w\w+\b',r'(?u)\b(?!\d+\b)\w+\b']
my_parameter_grid_by_name['bow__lowercase'] = [False,True]
my_parameter_grid_by_name['class__C'] = np.logspace(-4, 4, 9)

#my_scoring_metric_name = 'accuracy'
my_scoring_metric_name = 'roc_auc'

In [23]:
# prng = np.random.RandomState(0)

# valid_ids = prng.choice(np.arange(N), size=round(N/5))

# valid_indicators_N = np.zeros(N)
# valid_indicators_N[valid_ids] = -1
# my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_N)
y_train_clean = list(y_train_clean)
grid_searcher = sklearn.model_selection.GridSearchCV(
    my_bow_classifier_pipeline,
    my_parameter_grid_by_name,
    scoring=my_scoring_metric_name,
    #cv=my_splitter,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state),
    refit=True)
grid_searcher.fit(tr_text_list, y_train_clean)

c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('bow',
                                        CountVectorizer(lowercase=False,
                                                        max_df=1,
                                                        token_pattern='(?u)\\b[\\w-]+\\b')),
                                       ('class',
                                        LogisticRegression(max_iter=200,
                                                           random_state=42))]),
             param_grid={'bow__lowercase': [False, True],
                         'bow__max_df': [0.25, 0.5, 0.75, 1.0],
                         'bow__min_df': [1, 5, 10, 0.5, 0.75, 1.0],
                         'bow__ngram_range': [(1, 1), (1, 2)],
                         'bow__stop_words': [None, 'english'],
                         'bow__token_pattern': ['(?u)\\b[\\w-]+\\b',
                                                '(?u)\\b\\w\\w+\\b',
                                                '(?u)\\b(?!\\d+\\b)\\w+\\b'],
                         'class__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04])},
             scoring='roc_auc')

In [9]:
list(grid_searcher.cv_results_.keys())

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_bow__lowercase',
 'param_bow__max_df',
 'param_bow__min_df',
 'param_bow__ngram_range',
 'param_bow__stop_words',
 'param_bow__token_pattern',
 'param_class__C',
 'params',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'mean_test_score',
 'std_test_score',
 'rank_test_score']

In [24]:
gsearch_results_df = pd.DataFrame(grid_searcher.cv_results_).copy()

param_keys = ['param_bow__min_df','param_bow__max_df','param_bow__ngram_range','param_bow__stop_words','param_bow__token_pattern','param_bow__lowercase'
              , 'param_class__C']

# Rearrange row order so it is easy to skim
gsearch_results_df.sort_values(param_keys, inplace=True)
gsearch_results_df[param_keys + ['mean_test_score', 'rank_test_score']]

,param_bow__min_df,param_bow__max_df,param_bow__ngram_range,param_bow__stop_words,param_bow__token_pattern,param_bow__lowercase,param_class__C,mean_test_score,rank_test_score
369,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0001,NaN,2917
370,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0010,NaN,2917
371,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0100,NaN,2917
372,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.1000,NaN,2917
373,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,1.0000,NaN,2917
...,...,...,...,...,...,...,...,...,...
4819,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,1.0000,0.765899,1639
4820,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,10.0000,0.755650,1936
4821,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,100.0000,0.755386,1939
4822,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,1000.0000,0.759764,1814


In [31]:
test = gsearch_results_df[gsearch_results_df['mean_test_score'].isnull()]
test = test[test['param_bow__min_df']==0.75]
test[param_keys + ['mean_test_score', 'rank_test_score']]

,param_bow__min_df,param_bow__max_df,param_bow__ngram_range,param_bow__stop_words,param_bow__token_pattern,param_bow__lowercase,param_class__C,mean_test_score,rank_test_score
477,0.75,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0001,NaN,2917
478,0.75,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0010,NaN,2917
479,0.75,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0100,NaN,2917
480,0.75,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.1000,NaN,2917
481,0.75,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,1.0000,NaN,2917
...,...,...,...,...,...,...,...,...,...
5062,0.75,1.00,"(1, 2)",english,(?u)\b\w\w+\b,True,1.0000,NaN,2917
5063,0.75,1.00,"(1, 2)",english,(?u)\b\w\w+\b,True,10.0000,NaN,2917
5064,0.75,1.00,"(1, 2)",english,(?u)\b\w\w+\b,True,100.0000,NaN,2917
5065,0.75,1.00,"(1, 2)",english,(?u)\b\w\w+\b,True,1000.0000,NaN,2917


In [30]:
# Get the best-trained model (automatically refit on full training data)
best_model = grid_searcher.best_estimator_

# Best hyperparameters
best_params = grid_searcher.best_params_
print("Best Hyperparameters:", best_params)

# Best cross-validation score (average accuracy across folds)
best_score = grid_searcher.best_score_
print(f"Best CV AUROC: {best_score:.4f}")

Best Hyperparameters: {'bow__lowercase': False, 'bow__max_df': 0.5, 'bow__min_df': 1, 'bow__ngram_range': (1, 1), 'bow__stop_words': 'english', 'bow__token_pattern': '(?u)\\b(?!\\d+\\b)\\w+\\b', 'class__C': 1.0}
Best CV AUROC: 0.8346
